<figure> 
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Métodos ensamblados con mezla de datos: Bagging
    </center></span>

<figure> 
<center>
<img src="../Imagenes/Algerian_ensemble_Cairo_1932.jpg"  width="600" height="600" align="center"/>
<figcaption> Ensamble algeriano, Cairo, 1932</figcaption>
</center>
</figure>

Fuente <a href="https://commons.wikimedia.org/wiki/File:Algerian_ensemble_(Cairo_1932).jpg">AnonymousUnknown author</a>, Public domain, via Wikimedia Commons

## <span style="color:blue">Referencias</span>

1. [Breiman, Friedman, Olsen, Stone, Classification and Regression Trees, 1984](http://library.lol/main/26908B6EDA02CA4FAF25ADBF57A12B26)
1. [Kumar, A. and Jain, M., Ensemble learning for AI developers](http://library.lol/main/AC20329F24A966566561C7BF2A2A8529)
1. [Alvaro Montenegro y Daniel Montenegro, Inteligencia Artificial y Aprendizaje Profundo, 2022](https://github.com/AprendizajeProfundo/Diplomado)


## <span style="color:blue">Autores</span>

1. Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
1. Daniel  Montenegro, Msc, dammontenegrore@unal.edu.co


## <span style="color:blue">Asesora de Medios y  Marketing</span>

1. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [](#)

## <span style="color:blue">Introducción</span>

Charles Darwin descubrió esencialmente que las especies son menos vulnerables cuando tienen suficiente variedad genética. Es más probable que una especie se extinga si está expuesta solamente a un tipo de ambiente cerrado, cuando se producen cambios en el medio ambiente, por ejemplo causado por desastres naturales.

Por otro lado, especies que desarrollan una suficiente variedad genética, como consecuencia de estar expuestas a diferentes condiciones ambientales y de entorno, son más fuertes y resisten mejor los cambios ambientales o de entorno.

Traslademos estas ideas a la ciencia de datos. Cuando se entrena un modelo con todo el conjunto de datos, puede ocurrir que al colocar el modelo en producción, los nuevos datos no tengan la misma distribución de los datos de entrenamiento y el modelo deje de funcionar adecuadamente.

Las ideas en esta lección son: dividir los datos en distintos conjuntos y entrenar múltiples modelos sobre diferentes conjuntos de datos. las ventajas de aplicar estas ideas son:

* Al tener diferentes conjuntos de datos, las distribuciones de los datos de entrada cambian, así sea solamente un poco. Pero esto puede ser significativo, si por ejemplo, se pueden establecer patrones en los datos (grupos o clústers).
* Aplicar múltiples máquina de aprendizaje en una tarea rutinaria del científico de datos. La diferencia en este caso es que se aplican a diferentes conjuntos de datos, por lo que puede ocurrir que algunos modelos tengan mejores desempeños sobre subconjuntos de datos particulares.

Esta lección esta basada en [scikit-learn Ensemble methods](https://scikit-learn.org/stable/modules/ensemble.html#gradient-boosting) y [Kumar, A. and Jain, M., Ensemble learning for AI developers](http://library.lol/main/AC20329F24A966566561C7BF2A2A8529).

## <span style="color:blue">Métodos ensamblados</span>

Según la definición del diccionario de Cambridge, un "ensemble" se refiere a un grupo de elementos o personas que actúan juntos como un conjunto. Inicialmente utilizado en el contexto de músicos que tocan juntos regularmente, este término se aplica también en el aprendizaje automático, donde el "ensemble learning" implica combinar diversas técnicas de aprendizaje automático para mejorar la precisión de los modelos. Estos métodos combinan las salidas de múltiples modelos de aprendizaje automático de formas interesantes, similar a cómo un conjunto de músicos combina sus actuaciones individuales de diversas maneras para crear una gran composición. Como científico de datos, tu rol es ser el líder de orquesta o arquitecto que aprovecha las fortalezas de los modelos de aprendizaje automático individuales y los combina creativamente para lograr un modelo de aprendizaje automático de primera categoría.

Para comprender mejor los beneficios de los métodos de conjunto, consideremos otra analogía. Imagina que deseas invertir en el mercado de valores. Te interesa una acción en particular, pero no estás seguro de su perspectiva futura, por lo que decides buscar consejo. Consultas a un asesor financiero que tiene una precisión del 75% en sus predicciones correctas. Decides también consultar a otros asesores financieros que te ofrecen consejos similares. En el caso en que todos los asesores te sugieren comprar la acción, ¿cuál es la tasa de precisión de este consejo colectivo?
Más a menudo que no, el consejo colectivo de varios expertos supera la precisión de cualquier asesor individual, especialmente en situaciones financieras variadas. De manera similar, en el aprendizaje automático, los métodos de conjunto de múltiples modelos de aprendizaje automático tienden a tener un mejor rendimiento generalizado que cualquier modelo individual, especialmente en condiciones o casos diversos o a largo plazo. En este libro, te guiaremos a través de formas de combinar la salida de múltiples modelos de aprendizaje automático bajo el concepto de aprendizaje en conjunto. Las técnicas de aprendizaje en conjunto pueden dividirse en tres clases principales: mezcla de datos de entrenamiento, mezcla de combinaciones y mezcla de modelos. Intentaremos brevemente aumentar tu comprensión de cada una de estas clases.

<figure> 
<center>
<img src="../Imagenes/datosentrenamientomezclados.png"  width="600" height="600" align="center"/>
<figcaption> Mezclando datos de entrenamiento mediante bagging</figcaption>
</center>
</figure>

[Fuente](http://library.lol/main/AC20329F24A966566561C7BF2A2A8529): Kumar, A. & Jain, M. (2020). Mixing training data using bagging. Ensemble Learning for AI Developers. Página 2. Apress.

El objetivo de los métodos ensamblados es combinar las predicciones de varios estimadores base construidos con un algoritmo de aprendizaje dado para mejorar la generalización/robustez sobre un solo estimador.

Se suelen distinguir dos familias de métodos de conjunto:

En los métodos de promedio (**averging**), el principio fundamental es construir varios estimadores de forma independiente y luego promediar sus predicciones. En promedio, el estimador combinado suele ser mejor que cualquiera de los estimadores de base única porque se reduce su varianza.

Ejemplos: métodos de embolsado (**bagging**), bosques de árboles aleatorios (**ramdom forest**), ...

Por el contrario, en los métodos de impulso (**boosting**), los estimadores base se construyen secuencialmente y se intenta reducir el sesgo del estimador combinado. La motivación es combinar varios modelos débiles para producir un conjunto poderoso.

## <span style="color:blue">Bosques Aleatorios</span>

Hay grandes problemas con el uso de árboles de decisión. Obtener precisión suficiente para un conjunto de datos, necesita tener un árbol con mayor profundidad , pero a medida que aumenta la profundidad del árbol, comienza a enfrentarse a sobreajuste, lo que conduce a una menor precisión en el conjunto de datos de prueba.


Así que es mejor aceptar  una decisión menos precisa y menos profunda árbol  y no un árbol sobreajustado con más profundidad.

Una de las razones de este problema es que las variables utilizadas en la toma de decisiones puede no ser lo suficientemente discriminatorias.

Una forma de resolver este problema es tener múltiples árboles de decisión.
en lugar de uno. Cada árbol de decisión debe tener un conjunto diferente de variables 
o un subconjunto de datos de entrenamiento. Entonces, la salida de los árboles de decisión es combinado en un bosque aleatorio.

Como sugiere su nombre, un bosque aleatorio consiste en una colección de árboles de decisión, con cada árbol entrenado en un conjunto diferente de datos de entrenamiento.

El siguiente código construye  un bosque aleatorio en Python scikit-learn, con el conjunto de datos iris. Tomado del ejemplo en [Kumar, A. and Jain, M., Ensemble learning for AI developers](http://library.lol/main/AC20329F24A966566561C7BF2A2A8529).

In [3]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
train_X, test_X, train_Y, test_Y = train_test_split(X, y,
                    test_size = 0.1, random_state = 123)

forest = RandomForestClassifier(n_estimators=8)
forest = forest.fit(train_X, train_Y)
print('score: ', forest.score(test_X, test_Y))

rf_output = forest.predict(test_X)
print('predicciones: ', rf_output)

score:  1.0
predicciones:  [1 2 2 1 0 2 1 0 0 1 2 0 1 2 2]


Un bosque aleatorio de un conjunto de árboles de decisión ofrece lo mejor de ambos
mundos: mejor precisión con árboles de decisión menos profundos y menos posibilidades de
sobreajuste. 

Un bosque aleatorio es un ejemplo de conjuntosun ensamble de árboles de decisión. Tomamos
un solo modelo de aprendizaje automático (un árbol de decisiones) y lo entrenamos con una combinación de diferentes datos de entrenamiento y parámetros para hacer un modelo ensamblado.

## <span style="color:blue">Muestreo usando sckit-learn</span>

Para lo que sigue necesitaremos construir muestras de  los datos.

### <span style="color:#4CC9F0">Muestreo sin reemplazo </span>

In [4]:
from sklearn.utils import resample
import numpy as np

# Semilla para repetitibilidad
np.random.seed(123)

# datos para ser muestreados
data = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# Numero  de divisionss 
num_divisions = 2
list_of_data_divisions = []

for x in range(0, num_divisions):
    sample = resample(data, replace=False, n_samples=5)
    list_of_data_divisions.append(sample)

print('Muestras: ', list_of_data_divisions)

Muestras:  [[8, 1, 6, 7, 4], [4, 6, 5, 3, 8]]


### <span style="color:#4CC9F0">Muestreo con reemplazo </span>

In [6]:
from sklearn.utils import resample
import numpy as np

# Semilla para repetitibilidad
np.random.seed(123)

# datos para ser muestreados
data = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# Numero  de divisionss 
num_divisions = 3
list_of_data_divisions = []

for x in range(0, num_divisions):
    sample = resample(data, replace=True, n_samples=4)
    list_of_data_divisions.append(sample)

print('Muestras: ', list_of_data_divisions)

Muestras:  [[3, 3, 7, 2], [4, 7, 2, 1], [2, 1, 1, 4]]


## <span style="color:blue">Bagging. Agregación bootstrap</span>

Bagging es una forma abreviada de agregación de bootstrap. Es una técnica de ensamble que divide un conjunto de datos en $n$ muestras con reemplazo. Cada uno de las $n$ muestras divididas luego se entrenan por separado en $n$ máquinas de aprendizaje separadas. Luego, la salida de todos los modelos separados se combinan en una sola salida usando una votación.

Bagging consta de tres pasos: bootstrapping, entrenamiento y
agregación.

* Primero, el paso bootstrapping divide un conjunto de datos en $n$ muestras, con cada muestra un subconjunto de los datos de entrenamiento totales. Cada una de estas muestras tiene su muestreo realizado utilizando técnicas de muestreo con reemplazo. El muestreo con reemplazo asegura que el muestreo sea verdaderamente aleatorio. La composición de una muestra no depende de otra muestra

* El siguiente es el paso de entrenamiento, en el que entrena modelos individuales en estos muestras por separado. Este paso asegura que obtenga muchos de los relativamente débiles modelos de aprendizaje automático entrenados en cada muestra.

* El tercer paso es la agregación, en el que se combinan los resultados de todos los clasificadores débiles que usan métodos como una votación.

### <span style="color:#4CC9F0">Ejemplo de bagging paso a paso </span>

En el siguiente código se ilustran los tres pasos. La función *make_classification* es una herramienta de simulación que genera datos para problemas de clasificación bajo ciertas condiciones. Para los detalles consulte [sklearn.datasets.make_classification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html). Eejmplo tomado de [Kumar, A. and Jain, M., Ensemble learning for AI developers](http://library.lol/main/AC20329F24A966566561C7BF2A2A8529).

In [15]:
from sklearn.utils import resample
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np
from sklearn.metrics import accuracy_score

# crea los datos a ser muestreados
n_samples = 100
X,y = make_classification(n_samples=n_samples, n_features=4,
n_informative=2, n_redundant=0, random_state=0, shuffle=False)

# divide datos en entrenamineto y test
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size = 0.1, random_state = 123)

# Número de divisiones
num_divisions = 3
list_of_data_divisions = []

# Divide datos en subconjuntos
for x in range(0, num_divisions):
    X_train_sample, y_train_sample = resample(X_train, y_train,
    replace=True, n_samples=7)
    sample = [X_train_sample, y_train_sample]
    list_of_data_divisions.append(sample)

print('Primer elemento en la lista de divisiones: ', list_of_data_divisions[0])

# Entrena un Classifier por cada subconjunto de datos
learners = []
for data_division in list_of_data_divisions:
    data_x = data_division[0]
    data_y = data_division[1]
    decision_tree = tree.DecisionTreeClassifier()
    decision_tree.fit(data_x, data_y)
    learners.append(decision_tree)

# Combina la salida de todos los clasificadores usando votación
predictions = []
for i in range(len(y_test)):
    counts = [0 for _ in range(num_divisions)]
    for j , learner in enumerate(learners):
        prediction = learner.predict([X_test[i]])

        if prediction == 1:
            counts[j] = counts[j] + 1
    final_predictions = np.argmax(counts)
    predictions.append(final_predictions)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Primer elemento en la lista de divisiones:  [array([[ 1.83240861, -1.04999632, -0.04217145, -0.28688719],
       [-1.25732069, -3.19826339,  0.78632796, -0.4664191 ],
       [-0.82718247,  1.22006997, -1.93627981,  0.1887786 ],
       [ 1.34057624, -0.10789457,  0.76666318,  0.35629282],
       [ 1.23195055, -0.99510532, -0.94444626, -0.41004969],
       [ 0.86582546, -1.14855777,  1.0685094 , -0.4533858 ],
       [-1.05286598, -1.36672011, -0.63743703, -0.39727181]]), array([0, 0, 1, 1, 0, 0, 0])]
Accuracy: 0.8


In [21]:
print(np.asarray(predictions))
print(y_test)

[0 0 0 0 0 0 0 0 1 0]
[0 0 1 1 0 0 0 0 1 0]


## <span style="color:blue">Metaestimador bagging de scikit-learn</span>

En los algoritmos de ensamble, los métodos bagging forman una clase de algoritmos que crean varias instancias de un estimador de caja negra con subconjuntos aleatorios del conjunto de entrenamiento original y luego agregan sus predicciones individuales para formar una predicción final. 

Estos métodos se utilizan como una forma de reducir la varianza de un estimador base (por ejemplo, un árbol de decisión), introduciendo la aleatorización en su procedimiento de construcción y luego creando un conjunto a partir de él. 

En muchos casos, los métodos bagging constituyen una forma muy sencilla de mejorar las estimaciones con respecto a un único modelo, sin que sea necesario adaptar el algoritmo base básico subyacente. 

Como proporcionan una forma de reducir el sobreajuste, los métodos de embolsado funcionan mejor con modelos fuertes y complejos (p. ej., árboles de decisión completamente desarrollados), en contraste con los métodos de refuerzo que generalmente funcionan mejor con modelos débiles (p. ej., árboles de decisión poco profundos).

Los métodos de bagging vienen en muchos sabores, pero en su mayoría difieren entre sí por la forma en que se extraen los subconjuntos aleatorios del conjunto de entrenamiento:

* Cuando los subconjuntos aleatorios del conjunto de datos se extraen como subconjuntos aleatorios de las muestras, este algoritmo se conoce como pegado o`Pasting`.

* Cuando las muestras se extraen con reemplazo, el método se conoce como embolsado o `Bagging`.

* Cuando los subconjuntos aleatorios del conjunto de datos se dibujan como subconjuntos aleatorios de las características, el método se conoce como subespacios aleatorios o `random subspaces`.

* Finalmente, cuando los estimadores básicos se construyen sobre subconjuntos de muestras y características, el método se conoce como parches aleatorios o `Random Patches`.

En `scikit-learn`, los métodos de bagging se ofrecen como un metaestimador `BaggingClassifier` unificado (respectivamente `BaggingRegressor` para regresión), tomando como entrada un estimador base especificado por el usuario junto con parámetros que especifican la estrategia para dibujar subconjuntos aleatorios. 

En particular, 

* max_samples y max_features controlan el tamaño de los subconjuntos (en términos de muestras y funciones), 

* mientras que bootstrap y bootstrap_features controlan si las muestras y las funciones se dibujan con o sin reemplazo. 

Cuando se usa un subconjunto de las muestras disponibles, la precisión de la generalización se puede estimar con las muestras listas para usar configurando `oob_score=True`. 

### <span style="color:#4CC9F0">Ejemplo clasificador ensemble.Bagging.Classifier de scikit-learn </span>

Un clasificador de ensamble es un metaestimador de conjunto que ajusta los clasificadores base cada uno en subconjuntos aleatorios del conjunto de datos original y luego agrega sus predicciones individuales, usualment votando  para formar una predicción final.

Ejemplo tomado de [Kumar, A. and Jain, M., Ensemble learning for AI developers](http://library.lol/main/AC20329F24A966566561C7BF2A2A8529). 



In [26]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# crea los datos 
X, y = make_classification(n_samples=100, n_features=4,
                          n_informative=2, n_redundant=0,
                          random_state=0, shuffle=False)

# divide los datos en entrenamineto y test
X_train, X_test, y_train, y_test =  train_test_split(X, y,
                    test_size = 0.2, random_state = 123)

# hace la clasificación usandp bagging son SVC
clf = BaggingClassifier(base_estimator=SVC(),
        n_estimators=10, random_state=0).fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.85


### <span style="color:#4CC9F0">Ejemplo de regresión con  ensemble.Bagging.Regressor de scikit-learn </span>

Un regresor bagging es un metaestimador de ensamble que ajusta cada uno de los regresores base en subconjuntos aleatorios del conjunto de datos original y luego agrega sus predicciones individuales usualmente  promediando para formar una predicción final.
El ejemplo tomado de [scikit-learn]https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html) hace uso de la funcion *make_regression* que genra datos simulados para ejemplos de regresión.

In [27]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=100, n_features=4,
                        n_informative=2, n_targets=1,
                        random_state=0, shuffle=False)

regr = BaggingRegressor(base_estimator=SVR(),
                        n_estimators=10, random_state=0).fit(X, y)
regr.predict([[0, 0, 0, 0]])


array([-2.87202411])

### <span style="color:#4CC9F0">Estimador individual frente a bagging: descomposición de sesgo-varianza </span>

En este ejemplo, se compara la descomposición sesgo-varianza del error cuadrático medio esperado de un solo estimador con respecto a un conjunto de modelos mediante bagging. En la regresión, el error cuadrático medio esperado de un estimador se puede descomponer en sesgo, varianza y ruido. El sesgo mide cuánto difieren en promedio las predicciones del estimador de las predicciones del mejor estimador posible para el problema (es decir, el modelo de Bayes). La varianza mide la variabilidad de las predicciones del estimador cuando se ajusta a diferentes instancias aleatorias del mismo problema. Finalmente, el ruido mide la parte irreducible del error debida a la variabilidad en los datos. Los gráficos muestran cómo estas componentes cambian cuando se utiliza un solo árbol de decisión en comparación con un conjunto de modelos mediante bagging. Se observa que el sesgo aumenta ligeramente en el caso de bagging, pero la varianza disminuye significativamente, lo que resulta en un menor error cuadrático medio total. Esto respalda la eficacia del bagging para reducir la varianza y mejorar el rendimiento de los modelos.

<figure> 
<center>
<img src="../Imagenes/sphx_glr_plot_bias_variance_001.png"  width="600" height="600" align="center"/>
<figcaption> Estimador individual frente a bagging: descomposición de sesgo-varianza</figcaption>
</center>
</figure>

Fuente: [Scikit-learn](https://scikit-learn.org/stable/auto_examples/ensemble/plot_bias_variance.html#sphx-glr-auto-examples-ensemble-plot-bias-variance-py) 